In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objs as go

tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
print("Tesla Stock Data (First 5 Rows):")
print(tesla_data.head())

url_tesla = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_tesla = requests.get(url_tesla).text
soup_tesla = BeautifulSoup(html_tesla, "html.parser")
tables = soup_tesla.find_all("table")
for table in tables:
    if "Tesla Quarterly Revenue" in str(table):
        tesla_revenue_table = table
        break
tesla_revenue = pd.read_html(str(tesla_revenue_table))[0]
tesla_revenue.columns = ['Date', 'Revenue']
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != '']
tesla_revenue['Revenue'] = tesla_revenue['Revenue'].str.replace('$','').str.replace(',','')
tesla_revenue.dropna(inplace=True)
tesla_revenue['Revenue'] = pd.to_numeric(tesla_revenue['Revenue'])
tesla_revenue['Date'] = pd.to_datetime(tesla_revenue['Date'])
print("\nTesla Revenue Data (Last 5 Rows):")
print(tesla_revenue.tail())

gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
print("\nGameStop Stock Data (First 5 Rows):")
print(gme_data.head())

url_gme = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
html_gme = requests.get(url_gme).text
soup_gme = BeautifulSoup(html_gme, "html.parser")
tables = soup_gme.find_all("table")
for table in tables:
    if "GameStop Quarterly Revenue" in str(table):
        gme_revenue_table = table
        break
gme_revenue = pd.read_html(str(gme_revenue_table))[0]
gme_revenue.columns = ['Date', 'Revenue']
gme_revenue = gme_revenue[gme_revenue['Revenue'] != '']
gme_revenue['Revenue'] = gme_revenue['Revenue'].str.replace('$','').str.replace(',','')
gme_revenue.dropna(inplace=True)
gme_revenue['Revenue'] = pd.to_numeric(gme_revenue['Revenue'])
gme_revenue['Date'] = pd.to_datetime(gme_revenue['Date'])
print("\nGameStop Revenue Data (Last 5 Rows):")
print(gme_revenue.tail())

def make_graph(stock_data, revenue_data, stock_name):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=stock_data['Date'], y=stock_data['Close'], name="Stock Price"))
    fig.add_trace(go.Scatter(x=revenue_data['Date'], y=revenue_data['Revenue'], name="Revenue", yaxis="y2"))
    fig.update_layout(
        title=f"{stock_name} Stock Price and Revenue",
        xaxis_title="Date",
        yaxis=dict(title="Stock Price"),
        yaxis2=dict(title="Revenue", overlaying='y', side='right')
    )
    fig.show()

make_graph(tesla_data, tesla_revenue, "Tesla")

make_graph(gme_data, gme_revenue, "GameStop")
